# **30 Day Map Challenge 2022**
## *Day 2: Lines*
The goal for this second task is to create a simple map with highways of a specific city downloaded from OpenStreetMap. Highways are then plotted with their line thickness based on their hierarchical type.

In [ ]:
# import requested libraries
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
# Selecting the city for the map
place_name = "Milano, Italy"

In [ ]:
# Geocode the selected city and get its boundary
area = ox.geocode_to_gdf(place_name)

In [ ]:
# Check the data type
area

In [ ]:
# Download the road network from OSM and filter by chosen tags
filt = '["highway"~"motorway|trunk|primary|secondary|tertiary|residential"]'
G = ox.graph_from_place(place_name, network_type='drive', custom_filter=filt)

In [ ]:
# Converting the network in a geodataframe
gdf_nodes, gdf_edges = ox.graph_to_gdfs(
        G,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

In [ ]:
# Checking data contents
gdf_edges.head()

In [ ]:
# Plotting data
gdf_edges.plot()

In [ ]:
# Reprojecting data in the Italian CRS
highways = gdf_edges.to_crs(32632)
highways.plot()

In [ ]:
# Defining a function for reclassifying highways type (to be used later for line thickness plotting)
def label_highways (row):
   if row['highway'] == 'motorway' :
      return 2
   if row['highway'] == 'trunk' :
      return 1.5
   if row['highway'] == 'primary' :
      return 1
   if row['highway'] == 'secondary' :
      return 0.6
   if row['highway'] == 'tertiary' :
      return 0.4
   if row['highway'] == 'residential' :
      return 0.2

In [ ]:
# Applying the function to the geodataframe
highways.apply (lambda row: label_highways(row), axis=1)
highways['cat'] = highways.apply (lambda row: label_highways(row), axis=1)
highways.head()

In [ ]:
# Getting max and min coord values for the data
minx, miny, maxx, maxy = highways.geometry.total_bounds

In [ ]:
# Setting plot options
fig, ax = plt.subplots(figsize=(15, 15), facecolor='black')
ax = plt.gca()
_ = ax.axis('off')
highways.plot(color='white', edgecolor='none', linewidth=highways['cat'], ax=ax)
ax.add_artist(ScaleBar(1, box_color='black',color='white', font_properties={'family':'Special Elite', 'size': 'large'}, location='lower right'))
ax.annotate('30 Day Map Challenge - Day 2: Lines\nFederica Gaspari (@fede_gaspari)\nData: OpenStreetMap contributors\nTools: Python',font_properties={'family':'Special Elite', 'size': 'large'}, xy=(minx,miny), color='white')
plt.title('Find your way in '+ str(place_name) +'!',  fontname="Special Elite", color="white", fontsize="25")
#Defining filename and save image
filename = "images/Day022022_Lines.jpg"
fig.savefig(filename)
#plt.close(fig)